In [27]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pycountry_convert as pc

In [59]:
original_data = pd.read_excel('Data-file-Europe-Power-Sector-2020.xlsx', sheet_name = 'Data')
pivot_twh = pd.read_excel('Data-file-Europe-Power-Sector-2020.xlsx', sheet_name = 'Pivot, TWh', skiprows = 2, header = 1)
pivot_prod_percent = pd.read_excel('Data-file-Europe-Power-Sector-2020.xlsx', sheet_name = 'Pivot, % of production', skiprows = 2, header = 1)
carbon_intensities = pd.read_excel('Data-file-Europe-Power-Sector-2020.xlsx', sheet_name = 'Carbon intensities')

data = original_data.copy()

In [60]:
fossils = ['Coal', 'Hard Coal', 'Lignite', 'Gas', 'Other fossil', 'Nuclear']
renewables = ['Hydro', 'Wind', 'Solar', 'Bioenergy', 'Other renewables']
data['Kind of energy'] = data['Variable'].apply(
    lambda x: 'Renewable' if x in renewables else ('Fossil' if x in fossils else 'Other'))
data = data[data['Kind of energy'] != 'Other']

In [61]:
data.to_csv('energy_generation_per_source.csv')
pd.read_csv('energy_generation_per_source.csv').drop(columns=['Unnamed: 0'])

,Year,Area,Variable,Generation (TWh),Share of production (%),Change on last year (%),Change on last year (TWh),Kind of energy
0,2000,Austria,Coal,5.727000,9.568527,NaN,NaN,Fossil
1,2000,Austria,Hard Coal,4.435000,7.409886,NaN,NaN,Fossil
2,2000,Austria,Lignite,1.292000,2.158641,NaN,NaN,Fossil
3,2000,Austria,Gas,7.851000,13.117252,NaN,NaN,Fossil
4,2000,Austria,Other fossil,2.840859,4.746435,NaN,NaN,Fossil
...,...,...,...,...,...,...,...,...
6925,2020,United Kingdom,Hydro,6.596227,2.159324,12.597024,0.737966,Renewable
6926,2020,United Kingdom,Wind,73.775573,24.150984,14.332245,9.248218,Renewable
6927,2020,United Kingdom,Solar,13.353778,4.371459,2.959435,0.383837,Renewable
6928,2020,United Kingdom,Bioenergy,35.468198,11.610779,4.415895,1.500000,Renewable


In [62]:
data_per_year = {str(year): data[data['Year'] == year] for year in range(2000, 2021)}

energy_sources = ['Coal', 'Hard Coal', 'Lignite', 'Gas',
       'Nuclear', 'Hydro', 'Wind',
       'Solar', 'Bioenergy']
areas = data['Area'].unique()

energy_per_area_data = []
for year in range(2000, 2021):
    for area in areas:
        tmp = data_per_year[str(year)][data_per_year[str(year)]['Area'] == area]
        total_generation = tmp.groupby('Area')['Generation (TWh)'].sum()
        group = tmp.groupby('Kind of energy')
        generation_per_kind = group['Generation (TWh)'].sum()
#         display(generation_per_kind )
        
        total_energy_prod = tmp['Generation (TWh)'].sum()
        energy_sources_prod = [list(tmp[tmp['Variable'] == energy_source]['Generation (TWh)'])[0] for energy_source in energy_sources]
        energy_sources_share = [(energy_source_prod * 100 / total_energy_prod) for energy_source_prod in energy_sources_prod]
        
        share_of_prod = generation_per_kind * 100 / total_energy_prod
        
        line = [year, area, total_generation[0], generation_per_kind['Fossil'], generation_per_kind['Renewable'], share_of_prod[0], share_of_prod[1]]
        line += energy_sources_prod + energy_sources_share
        energy_per_area_data.append(line)

columns = ['Year','Area', 'Total energy generation (TWh)', 'Fossil energy generation (TWh)', 'Renewable energy generation (TWh)', 'Fossil energy share of generation (%)', 'Renewable energy share of generation (%)']
columns += [energy_source + ' generation (TWh)' for energy_source in energy_sources] + [energy_source + ' energy share of generation (%)' for energy_source in energy_sources]
energy_per_area = pd.DataFrame(data=energy_per_area_data,columns = columns)
energy_per_area

,Year,Area,Total energy generation (TWh),Fossil energy generation (TWh),Renewable energy generation (TWh),Fossil energy share of generation (%),Renewable energy share of generation (%),Coal generation (TWh),Hard Coal generation (TWh),Lignite generation (TWh),...,Bioenergy generation (TWh),Coal energy share of generation (%),Hard Coal energy share of generation (%),Lignite energy share of generation (%),Gas energy share of generation (%),Nuclear energy share of generation (%),Hydro energy share of generation (%),Wind energy share of generation (%),Solar energy share of generation (%),Bioenergy energy share of generation (%)
0,2000,Austria,65.579473,22.145859,43.433614,33.769498,66.230502,5.727000,4.435000,1.292000,...,1.528279,8.732916,6.762787,1.970129,11.971734,0.000000,63.793364,0.101791,0.004924,2.330423
1,2000,Belgium,95.689000,94.645000,1.044000,98.908966,1.091034,12.916000,12.916000,0.000000,...,0.568000,13.497894,13.497894,0.000000,16.696799,50.326579,0.480724,0.016721,0.000000,0.593590
2,2000,Bulgaria,57.545000,54.914000,2.631000,95.427926,4.572074,16.941000,2.972000,13.969000,...,0.000000,29.439569,5.164654,24.274915,3.322617,31.589191,4.572074,0.000000,0.000000,0.000000
3,2000,Croatia,12.813900,6.360000,6.453900,49.633601,50.366399,1.551000,1.546000,0.005000,...,0.001000,12.104043,12.065023,0.039020,12.260124,0.000000,50.358595,0.000000,0.000000,0.007804
4,2000,Cyprus,3.370000,3.370000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2020,Slovakia,30.304375,22.869153,7.435221,75.464858,24.535142,1.976606,0.748827,1.227779,...,1.925314,6.522510,2.471020,4.051490,12.420880,50.944473,15.681428,0.032854,1.994519,6.353253
626,2020,Slovenia,21.341835,15.694154,5.647681,73.537041,26.462959,4.377792,0.408638,3.969154,...,0.285030,20.512726,1.914728,18.597999,2.799253,29.653136,23.742028,0.029205,1.356179,1.335546
627,2020,Spain,270.030643,155.405145,114.625497,57.550930,42.449070,6.295944,6.295944,0.000000,...,6.792585,2.331566,2.331566,0.000000,25.405739,21.592704,11.815222,20.422445,7.686268,2.515487
628,2020,Sweden,166.750708,54.217456,112.533252,32.514078,67.485922,0.335000,0.335000,0.000000,...,10.689012,0.200899,0.200899,0.000000,0.230884,30.120085,44.469793,16.361877,0.244077,6.410175


In [67]:
energy_per_area.to_csv('energy_per_area.csv')
pd.read_csv('energy_per_area.csv').drop(columns=['Unnamed: 0'])

,Year,Area,Total energy generation (TWh),Fossil energy generation (TWh),Renewable energy generation (TWh),Fossil energy share of generation (%),Renewable energy share of generation (%),Coal generation (TWh),Hard Coal generation (TWh),Lignite generation (TWh),...,Bioenergy generation (TWh) (log_10),Coal energy share of generation (%) (log_10),Hard Coal energy share of generation (%) (log_10),Lignite energy share of generation (%) (log_10),Gas energy share of generation (%) (log_10),Nuclear energy share of generation (%) (log_10),Hydro energy share of generation (%) (log_10),Wind energy share of generation (%) (log_10),Solar energy share of generation (%) (log_10),Bioenergy energy share of generation (%) (log_10)
0,2000,Austria,65.579473,22.145859,43.433614,33.769498,66.230502,5.727000,4.435000,1.292000,...,0.184203,0.941159,0.830126,0.294495,1.078157,-inf,1.804776,-0.992291,-2.307700,0.367435
1,2000,Belgium,95.689000,94.645000,1.044000,98.908966,1.091034,12.916000,12.916000,0.000000,...,-0.245652,1.130266,1.130266,-inf,1.222633,1.701797,-0.318104,-1.776742,-inf,-0.226514
2,2000,Bulgaria,57.545000,54.914000,2.631000,95.427926,4.572074,16.941000,2.972000,13.969000,...,-inf,1.468931,0.713041,1.385158,0.521480,1.499539,0.660113,-inf,-inf,-inf
3,2000,Croatia,12.813900,6.360000,6.453900,49.633601,50.366399,1.551000,1.546000,0.005000,...,-3.000000,1.082930,1.081528,-1.408711,1.088495,-inf,1.702074,-inf,-inf,-2.107681
4,2000,Cyprus,3.370000,3.370000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2020,Slovakia,30.304375,22.869153,7.435221,75.464858,24.535142,1.976606,0.748827,1.227779,...,0.284501,0.814415,0.392876,0.607615,1.094152,1.707097,1.195386,-1.483414,0.299838,0.802996
626,2020,Slovenia,21.341835,15.694154,5.647681,73.537041,26.462959,4.377792,0.408638,3.969154,...,-0.545109,1.312023,0.282107,1.269466,0.447042,1.472071,1.375518,-1.534536,0.132317,0.125659
627,2020,Spain,270.030643,155.405145,114.625497,57.550930,42.449070,6.295944,6.295944,0.000000,...,0.832035,0.367648,0.367648,-inf,1.404932,1.334307,1.072442,1.310108,0.885716,0.400622
628,2020,Sweden,166.750708,54.217456,112.533252,32.514078,67.485922,0.335000,0.335000,0.000000,...,1.028938,-0.697023,-0.697023,-inf,-0.636607,1.478856,1.648065,1.213833,-0.612473,0.806870


In [64]:
display(energy_per_area[energy_per_area['Year'] == 2020][['Area', 'Year', 'Renewable energy generation (TWh)']])
data[(data['Year'] == 2020) & (data['Kind of energy'] == 'Renewable')].groupby('Area').sum()

,Area,Year,Renewable energy generation (TWh)
600,Austria,2020,55.084913
601,Belgium,2020,22.442189
602,Bulgaria,2020,7.497195
603,Croatia,2020,8.152167
604,Cyprus,2020,0.513849
605,Czech Republic,2020,9.753075
606,Denmark,2020,22.718575
607,Estonia,2020,2.235610
608,EU-27,2020,1054.334436
609,EU27+1,2020,1183.537511


,Year,Generation (TWh),Share of production (%),Change on last year (%),Change on last year (TWh)
Area,,,,,
Austria,10100,55.084913,79.267640,-22.739013,-0.042862
Belgium,10100,22.442189,26.119167,39.487327,3.677651
Bulgaria,10100,7.497195,18.591082,22.692265,0.324293
Croatia,10100,8.152167,64.070524,-14.484371,-0.145854
Cyprus,10100,0.513849,11.186482,7.285415,0.017483
Czech Republic,10100,9.753075,12.088354,-80.091773,0.480541
Denmark,10100,22.718575,78.208881,20.741083,0.105748
EU-27,10100,1054.334436,38.204183,30.812198,77.186458
EU27+1,10100,1183.537511,38.611929,31.523181,89.056480


In [65]:
columns = energy_per_area.columns
print(columns)
for i in range(2, len(columns)):
   # if '%' not in columns[i]:
    energy_per_area[columns[i] + ' (log_10)']= energy_per_area[columns[i]].apply(np.log10)

print(energy_per_area.columns)
display(energy_per_area)


Index(['Year', 'Area', 'Total energy generation (TWh)',
       'Fossil energy generation (TWh)', 'Renewable energy generation (TWh)',
       'Fossil energy share of generation (%)',
       'Renewable energy share of generation (%)', 'Coal generation (TWh)',
       'Hard Coal generation (TWh)', 'Lignite generation (TWh)',
       'Gas generation (TWh)', 'Nuclear generation (TWh)',
       'Hydro generation (TWh)', 'Wind generation (TWh)',
       'Solar generation (TWh)', 'Bioenergy generation (TWh)',
       'Coal energy share of generation (%)',
       'Hard Coal energy share of generation (%)',
       'Lignite energy share of generation (%)',
       'Gas energy share of generation (%)',
       'Nuclear energy share of generation (%)',
       'Hydro energy share of generation (%)',
       'Wind energy share of generation (%)',
       'Solar energy share of generation (%)',
       'Bioenergy energy share of generation (%)'],
      dtype='object')
Index(['Year', 'Area', 'Total energy genera

,Year,Area,Total energy generation (TWh),Fossil energy generation (TWh),Renewable energy generation (TWh),Fossil energy share of generation (%),Renewable energy share of generation (%),Coal generation (TWh),Hard Coal generation (TWh),Lignite generation (TWh),...,Bioenergy generation (TWh) (log_10),Coal energy share of generation (%) (log_10),Hard Coal energy share of generation (%) (log_10),Lignite energy share of generation (%) (log_10),Gas energy share of generation (%) (log_10),Nuclear energy share of generation (%) (log_10),Hydro energy share of generation (%) (log_10),Wind energy share of generation (%) (log_10),Solar energy share of generation (%) (log_10),Bioenergy energy share of generation (%) (log_10)
0,2000,Austria,65.579473,22.145859,43.433614,33.769498,66.230502,5.727000,4.435000,1.292000,...,0.184203,0.941159,0.830126,0.294495,1.078157,-inf,1.804776,-0.992291,-2.307700,0.367435
1,2000,Belgium,95.689000,94.645000,1.044000,98.908966,1.091034,12.916000,12.916000,0.000000,...,-0.245652,1.130266,1.130266,-inf,1.222633,1.701797,-0.318104,-1.776742,-inf,-0.226514
2,2000,Bulgaria,57.545000,54.914000,2.631000,95.427926,4.572074,16.941000,2.972000,13.969000,...,-inf,1.468931,0.713041,1.385158,0.521480,1.499539,0.660113,-inf,-inf,-inf
3,2000,Croatia,12.813900,6.360000,6.453900,49.633601,50.366399,1.551000,1.546000,0.005000,...,-3.000000,1.082930,1.081528,-1.408711,1.088495,-inf,1.702074,-inf,-inf,-2.107681
4,2000,Cyprus,3.370000,3.370000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2020,Slovakia,30.304375,22.869153,7.435221,75.464858,24.535142,1.976606,0.748827,1.227779,...,0.284501,0.814415,0.392876,0.607615,1.094152,1.707097,1.195386,-1.483414,0.299838,0.802996
626,2020,Slovenia,21.341835,15.694154,5.647681,73.537041,26.462959,4.377792,0.408638,3.969154,...,-0.545109,1.312023,0.282107,1.269466,0.447042,1.472071,1.375518,-1.534536,0.132317,0.125659
627,2020,Spain,270.030643,155.405145,114.625497,57.550930,42.449070,6.295944,6.295944,0.000000,...,0.832035,0.367648,0.367648,-inf,1.404932,1.334307,1.072442,1.310108,0.885716,0.400622
628,2020,Sweden,166.750708,54.217456,112.533252,32.514078,67.485922,0.335000,0.335000,0.000000,...,1.028938,-0.697023,-0.697023,-inf,-0.636607,1.478856,1.648065,1.213833,-0.612473,0.806870


In [66]:
columns = energy_per_area.columns.to_numpy()[2:]
dropdown_options = [{'label':columns[i], 'value': i} for i in range(len(columns))]
dropdown_options

[{'label': 'Total energy generation (TWh)', 'value': 0},
 {'label': 'Fossil energy generation (TWh)', 'value': 1},
 {'label': 'Renewable energy generation (TWh)', 'value': 2},
 {'label': 'Fossil energy share of generation (%)', 'value': 3},
 {'label': 'Renewable energy share of generation (%)', 'value': 4},
 {'label': 'Coal generation (TWh)', 'value': 5},
 {'label': 'Hard Coal generation (TWh)', 'value': 6},
 {'label': 'Lignite generation (TWh)', 'value': 7},
 {'label': 'Gas generation (TWh)', 'value': 8},
 {'label': 'Nuclear generation (TWh)', 'value': 9},
 {'label': 'Hydro generation (TWh)', 'value': 10},
 {'label': 'Wind generation (TWh)', 'value': 11},
 {'label': 'Solar generation (TWh)', 'value': 12},
 {'label': 'Bioenergy generation (TWh)', 'value': 13},
 {'label': 'Coal energy share of generation (%)', 'value': 14},
 {'label': 'Hard Coal energy share of generation (%)', 'value': 15},
 {'label': 'Lignite energy share of generation (%)', 'value': 16},
 {'label': 'Gas energy share 

In [51]:
fossil_energies = ['Fossil', 'Coal', 'Hard Coal', 'Lignite', 'Gas', 'Other fossil', 'Nuclear']
renewable_energies = ['Renewable', 'Hydro', 'Wind and solar', 'Wind', 'Solar', 'Bioenergy', 'Other renewables']
colorscales = {col: ('Greens' if any(map((lambda energie: energie in col), renewable_energies)) else 'Reds') for col in columns}
colorscales['Total energy generation (TWh)'] = 'Blues'
colorscales

{'Total energy generation (TWh)': 'Blues',
 'Fossil energy generation (TWh)': 'Reds',
 'Renewable energy generation (TWh)': 'Greens',
 'Fossil energy share of generation (%)': 'Reds',
 'Renewable energy share of generation (%)': 'Greens',
 'Coal generation (TWh)': 'Reds',
 'Hard Coal generation (TWh)': 'Reds',
 'Lignite generation (TWh)': 'Reds',
 'Gas generation (TWh)': 'Reds',
 'Nuclear generation (TWh)': 'Reds',
 'Hydro generation (TWh)': 'Greens',
 'Wind generation (TWh)': 'Greens',
 'Solar generation (TWh)': 'Greens',
 'Bioenergy generation (TWh)': 'Greens',
 'Coal energy share of generation (%)': 'Reds',
 'Hard Coal energy share of generation (%)': 'Reds',
 'Lignite energy share of generation (%)': 'Reds',
 'Gas energy share of generation (%)': 'Reds',
 'Nuclear energy share of generation (%)': 'Reds',
 'Hydro energy share of generation (%)': 'Greens',
 'Wind energy share of generation (%)': 'Greens',
 'Solar energy share of generation (%)': 'Greens',
 'Bioenergy energy share of 

In [148]:
def update_line_plot(y_axis)
    return px.line(energy_per_area, x="Year", y="Renewables energy production %", color='Area')
update_line_plot("Renewables energy production %")

In [41]:


def create_map(df, year, areas):
    df = df[df['Year'] == year]
    world_data = df.groupby('Area').sum()
    world_data['Year'] = year
    world_data = world_data.reset_index()
    
    unwanted_area = ['EU-27', 'EU27+1']
    world_data = world_data[world_data['Area'].isin(areas)]
    display(world_data)
    
    
    # world_data['iso_alpha3'] = world_data['Area'].apply(lambda x: pc.country_name_to_country_alpha3(x))
    
    # fig = go.Figure(data=go.Choropleth(
    #     locations = world_data['iso_alpha3'],
    #     # z = world_data['Generation (TWh)'],
    #     # text = world_data['Area'],
    #     # colorscale = 'Reds',
    #     # autocolorscale=False,
    #     # reversescale=True,
    #     # marker_line_color='darkgray',
    #     # marker_line_width=0.5,
    #     # colorbar_title = 'Generation<br>TWh'
    # ))
    
    # fig.update_geos(
    #     projection_scale=4.5,
    #     center={'lat':52, 'lon': 11.977321},
    #     #projection={"type": "mercator"},
    # )
    # fig.update_layout(autosize=True ,margin={"r":0,"t":0,"l":0,"b":0})


    # # fig.update_layout(
    # #     title_text = str(year) + ' Energy generation',
    # #     geo=dict(
    # #         showframe=False,
    # #         showcoastlines=False,
    # #         projection_type='equirectangular'
    # #     ),
    # #     annotations = [dict(
    # #         x=0.55,
    # #         y=0.1,
    # #         xref='paper',
    # #         yref='paper',
    # #         text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
    # #             CIA World Factbook</a>',
    # #         showarrow = False
    # #     )]
    # # )
    
    # return fig

create_map(data, 2020, ['France', 'Belgium'])

,Area,Year,Generation (TWh),Share of production (%),Change on last year (%),Change on last year (TWh)
1,Belgium,2020,86.013598,100.106259,14.262622,-4.381144
11,France,2020,532.787203,100.808967,1.178306,-36.595828


In [42]:
df = data[data['Year'] == 2002]
world_data = df.groupby('Area').sum()
world_data['Year'] = 2002
world_data = world_data.reset_index()
    
unwanted_area = ['EU-27', 'EU27+1']
world_data = world_data[~world_data['Area'].isin(unwanted_area)]

import geojson
# geojson_file = 'europe.geojson'
geojson_file = 'custom.geo.json'
    
with open(geojson_file, "r", encoding="utf-8") as f:
    geo_world = geojson.load(f)

# Instanciating necessary lists
found = []
missing = []
countries_geo = []

# For simpler acces, setting "zone" as index in a temporary dataFrame
tmp = world_data.set_index('Area')

# Looping over the custom GeoJSON file
for country in geo_world['features']:
    
    # Country name detection
    country_name = country['properties']['name'] 
    
    # Checking if that country is in the dataset
    if country_name in tmp.index:
        
        # Adding country to our "Matched/found" countries
        found.append(country_name)
        
        # Getting information from both GeoJSON file and dataFrame
        geometry = country['geometry']
        
        # Adding 'id' information for further match between map and data 
        countries_geo.append({
            'type': 'Feature',
            'geometry': geometry,
            'id':country_name
        })
        
    # Else, adding the country to the missing countries
    else:
        missing.append(country_name)

# Displaying metrics
print(f'Countries found    : {len(found)}')
print(f'Countries not found: {len(missing)}')
geo_world_ok = {'type': 'FeatureCollection', 'features': countries_geo}
print('missing', missing)
print('found', found)
# Create the log count column
world_data['count_color'] = world_data['Generation (TWh)'].apply(np.log10)

# Get the maximum value to cap displayed values
max_log = world_data['count_color'].max()
max_val = int(max_log) + 1

# Prepare the range of the colorbar
values = [i for i in range(max_val)]
ticks = [10**i for i in values]


# Create figure
fig = px.choropleth(
    world_data,
    geojson=geo_world_ok,
    locations='Area',
    color=world_data['count_color'],
    range_color=(0, world_data['count_color'].max()),
)

# Define layout specificities
fig.update_layout(
    margin={'r':0,'t':0,'l':0,'b':0},
    coloraxis_colorbar={
        'title':'Confirmed people',
        'tickvals':values,
        'ticktext':ticks        
    }
)

# Display figure
fig.show()

ModuleNotFoundError: No module named 'geojson'

In [86]:
def production_share_sunburst(data, year, area):
    unwanted_variables = ['Demand', 'Production', 'Net imports', 'Fossil', 'Renewables']
    df = data[(data['Year'] == year) & (data['Area'] == area)]
    df = df[~df['Variable'].isin(unwanted_variables)]
    generation_per_kind = df.groupby(by='Kind of energy')['Generation (TWh)'].sum()
    df['Kind of Energy generation (TWh)'] = df['Kind of energy'].apply(lambda x: generation_per_kind[x])
    display(df)



    fig = px.sunburst(df, path=['Kind of energy', 'Variable'],
                        values='Share of production (%)',
                        hover_name='Variable',
                        hover_data=['Generation (TWh)', 'Change on last year (%)', 'Change on last year (TWh)'])

    fig2 =go.Figure(go.Sunburst(
        branchvalues = 'total',
        labels=fig['data'][0]['labels'].tolist(),
        parents=fig['data'][0]['parents'].tolist(),
        values=fig['data'][0]['values'].tolist(),
        ids=fig['data'][0]['ids'].tolist()
                                )
                   )
    return fig

production_share_sunburst(data, 2020, 'EU27+1')

,Year,Area,Variable,Generation (TWh),Share of production (%),Change on last year (%),Change on last year (TWh),Kind of energy,Kind of Energy generation (TWh)
10354,2020,EU27+1,Coal,370.049998,12.072574,-19.931844,-92.118754,Fossil,2251.724581
10355,2020,EU27+1,Hard Coal,169.881362,5.542238,-22.024492,-47.983666,Fossil,2251.724581
10356,2020,EU27+1,Lignite,200.168636,6.530336,-18.065663,-44.135087,Fossil,2251.724581
10357,2020,EU27+1,Gas,656.241552,21.409336,-5.880167,-40.998900,Fossil,2251.724581
10358,2020,EU27+1,Other fossil,118.461917,3.864722,-5.007324,-6.244452,Fossil,2251.724581
10359,2020,EU27+1,Nuclear,736.921115,24.041440,-10.409942,-85.626758,Fossil,2251.724581
10361,2020,EU27+1,Hydro,358.163336,11.684782,8.529249,28.147842,Renewable,1810.829239
10362,2020,EU27+1,Wind and solar,627.291728,20.464872,10.727655,60.774059,Renewable,1810.829239
10363,2020,EU27+1,Wind,470.218006,15.340472,9.784277,41.907124,Renewable,1810.829239
10364,2020,EU27+1,Solar,157.073723,5.124400,13.651237,18.866935,Renewable,1810.829239


In [106]:
%%writefile /tmp/dash1.py
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from dash import html
from dash import dcc
import pycountry_convert as pc
import dash
# -*- coding: utf-8 -*-

pd.options.plotting.backend = "plotly"

# ========================================================================================
#
#
#                            DATA LOADING AND VARIABLE DECLARATIONS
#
#
# ========================================================================================

data = pd.read_csv('energy_generation_per_source.csv').drop(columns=['Unnamed: 0'])
energy_per_area = pd.read_csv('energy_per_area.csv').drop(columns=['Unnamed: 0'])
data_per_year = {str(year): data[data['Year'] == year]
                 for year in range(2000, 2021)}

areas = data['Area'].unique()

area_minus_europe = [
    countries for countries in areas if countries not in ['EU-27', 'EU27+1']]


columns = energy_per_area.columns.to_numpy()[2:]
dropdown_options = [{'label': columns[i], 'value': i}
                    for i in range(len(columns)) if '(log_10)' not in columns[i]]
(dropdown_options[0], dropdown_options[4]) = (dropdown_options[4], dropdown_options[0])
y_axis = {str(i): columns[i] for i in range(len(columns))}

fossil_energies = ['Fossil', 'Coal', 'Hard Coal',
                   'Lignite', 'Gas', 'Other fossil', 'Nuclear']
renewable_energies = ['Renewable', 'Hydro', 'Wind and solar',
                      'Wind', 'Solar', 'Bioenergy', 'Other renewables']
colorscales = {col: ('Greens' if any(map(
    (lambda energie: energie in col), renewable_energies)) else 'Reds') for col in columns}
colorscales['Total energy generation (TWh)'] = 'Blues'
colorscales['Total energy generation (TWh) (log_10)'] = 'Blues'


class EuropeEnergyGeneration():
    START = 'Start'
    STOP = 'Stop'

    def __init__(self):
        self.colorscales = colorscales
        self.dropdown_options = dropdown_options
        self.data_per_year = data_per_year
        self.energy_per_area = energy_per_area
        self.df = data
        self.app = dash.Dash()
        self.years = [i for i in range(2000, 2021)]
        self.y_axis = y_axis
        self.is_autoslider_activated = False
        self.colors = {
            'thirdly': '#D3D3D3',
            'secondary': '#111111',
            'background': '#111111',
            'text': '#7FDBFF'
        }

        # ========================================================================================
        #
        #
        #                            APP LAYOUT
        #
        #
        # ========================================================================================

        self.app.layout = html.Div(children=[
            html.H1(children='Energy generation in the European Union from 2000 to 2020',
                    style={
                        'textAlign': 'center'
                    }),

            html.Div(children=[
                html.Div(
                    dcc.Dropdown(
                        dropdown_options,
                        value=4,
                        id='y-axis',
                        searchable=True,
                        style = {
                            'backgroundColor': self.colors['thirdly'],
                            'color': self.colors['text']
                        }
                    ),
                    style={'width': '35%',
                           'display': 'inline flow-root',
                          }
                ),

                html.Div(
                        dcc.RadioItems(options=[
                            {'label': 'Linear', 'value': 0},
                            {'label': 'Log', 'value': 1}
                        ],
                            value=0,
                            id='scale',
                            style={
                                    'display': 'flex'
                                  }
                        ),
                    style={
                        'paddingLeft': '4%',
                        'width': '33%',
                        'display': 'flex',
                        'flexDirection': 'row',
                        'alignItems': 'center'
                    }
                )],
                     style={
                        'paddingLeft': '30%',
                        'display': 'flex'
                     }
            ),

            html.Div(children=[
                html.Div(children=[
                    html.Div(
                        dcc.Graph(id='country-sunburst',
                                  style={
                                    'display': 'inline-block'
                                    })
                    ),
                    html.Div(
                        dcc.Graph(
                            id='europe27+1-sunburst',
                            style={
                                 'display': 'inline flow-root list-item'
                            })
                    ),
                ],
                style={}
                ),
                html.Div(children=[
                    html.Div(
                        dcc.Graph(
                            id='Europe-Map',
                            style={
                                'width': '100%',
                                'height': '80%',
                                'display': 'inline-block'
                            }
                        ),
                        style={
                            'width': '100%',
                            'height': '100%',
                        }
                    ),
                    html.Div(
                        dcc.Graph(id='evolution-line-plot',
                                  style={
                                      'width': '100%',
                                      'height': '80%',
                                      'display': 'inline flow-root list-item'
                                  }),
                        style={
                        }
                    ),
                ])
            ],
            style={
                'backgroundColor': self.colors['secondary'],
                'color': self.colors['text'],
                'display':'flex',
                'alignItems':'center' 
            }),

            html.Div(children=[
                html.Div(children=[
                    html.Button(
                        self.STOP,
                        id='wps-button-start-stop',
                        style={'display': 'inline-block'}
                    ),
                    html.Div(
                        dcc.Slider(
                            id='wps-crossfilter-year-slider',
                            min=self.years[0],
                            max=self.years[-1],
                            step=1,
                            value=2019,
                            marks={str(year): {'label': str(year),
                                   'style': {'color': self.colors['text'],
                                             'backgroundColor': self.colors['secondary']}}
                                   for year in self.years}
                        ),
                        style={'display': 'inline-block',
                               'width': "90%",
                              }
                    ),
                    dcc.Interval(            # fire a callback periodically
                        id='wps-auto-stepper',
                        disabled=self.is_autoslider_activated,
                        interval=1000,       # in milliseconds
                        max_intervals=-1,  # start running
                        n_intervals=0
                    ),
                ],
                style={
                    'display':'flex'
                }),
                html.Div(children=[
                    html.H5("Click on a the cross next to a country to unselect it from the map/ select it from the dropdown",
                    style={
                        'textAlign':'center'
                    }),
                    dcc.Dropdown(
                        area_minus_europe, area_minus_europe, multi=True, id='selected-areas-for-map',
                        style = {
                            'backgroundColor': self.colors['secondary'],
                            'color': self.colors['text']
                        }
                    )],
                    style={
                        'display':'flex',
                        'flexDirection':'column'
                    }
                ),
                dcc.Markdown("""
   #### À propos

   * Inspiration initiale : [Conférence de Hans Rosling](https://www.ted.com/talks/hans_rosling_new_insights_on_poverty)
   * [Version Plotly](https://plotly.com/python/v3/gapminder-example/)
   * Données : [DataWorld, d'après les données de Ember-Climate](https://data.world/makeovermonday/2021w5)
   * (c) 2022 Yacine Anane && Charli De Luca
   """),

            ])
        ],
        style={
            'backgroundColor': self.colors['background'],
            'color': self.colors['text']})

        # ========================================================================================
        #
        #
        #                            CALLBACKS
        #
        #
        # ========================================================================================
        self.app.callback(
            dash.dependencies.Output('Europe-Map', 'figure'),
            dash.dependencies.Input('wps-crossfilter-year-slider', 'value'),
            dash.dependencies.Input('y-axis', 'value'),
            dash.dependencies.Input('selected-areas-for-map', 'value'),
            dash.dependencies.Input('scale', 'value')
        )(self.create_map)

        self.app.callback(
            dash.dependencies.Output('country-sunburst', 'figure'),
            dash.dependencies.Input('Europe-Map', 'clickData'),
            dash.dependencies.Input('wps-crossfilter-year-slider', 'value')
        )(self.update_sunburst_country)

        self.app.callback(
            dash.dependencies.Output('europe27+1-sunburst', 'figure'),
            dash.dependencies.Input('wps-crossfilter-year-slider', 'value')
        )(self.update_sunburst_europe)

        self.app.callback(
            dash.dependencies.Output('evolution-line-plot', 'figure'),
            dash.dependencies.Input('y-axis', 'value'),
            dash.dependencies.Input('scale', 'value')
        )(self.update_line_plot)

        self.app.callback(
            dash.dependencies.Output('wps-crossfilter-year-slider', 'value'),
            dash.dependencies.Input('wps-auto-stepper', 'n_intervals'),
            [dash.dependencies.State('wps-crossfilter-year-slider', 'value'),
             dash.dependencies.State('wps-button-start-stop', 'children')]
        )(self.on_interval)

        self.app.callback(
            dash.dependencies.Output('wps-button-start-stop', 'children'),
            dash.dependencies.Input('wps-button-start-stop', 'n_clicks'),
            dash.dependencies.State('wps-button-start-stop', 'children')
        )(self.button_on_click)

        self.app.callback(
            dash.dependencies.Output('wps-auto-stepper', 'max_interval'),
            [dash.dependencies.Input('wps-button-start-stop', 'children')]
        )(self.run_movie)

        self.app.callback(
            dash.dependencies.Output('text-sunburst-country', 'children'),
            dash.dependencies.Input('Europe-Map', 'clickData')
        )(self.update_country_name)

    def create_map(self, year, y_axis_value, selected_areas, scale):
        world_data = self.energy_per_area[self.energy_per_area['Area'].isin(
            selected_areas)]

       
        world_data = world_data[world_data['Year'] == year]
        world_data['iso_alpha3'] = world_data['Area'].apply(
            lambda x: pc.country_name_to_country_alpha3(x))
        
        color_column = self.y_axis[str(y_axis_value)] if scale == 0 else (self.y_axis[str(y_axis_value)] + ' (log_10)')
        zmax = world_data[color_column].max()
        zmin = world_data[color_column].min()

#         world_data['count_color'] = world_data['Generation (TWh)'].apply(np.log10)
#         # Get the maximum value to cap displayed values
#         max_log = world_data['count_color'].max()
#         max_val = int(max_log) + 1

#         # Prepare the range of the colorbar
#         values = [i for i in range(max_val)]
#         ticks = [10**i for i in values]

        fig = go.Figure(data=go.Choropleth(
            locations=world_data['iso_alpha3'],
            z=world_data[color_column],
            text=world_data['Area'],
            #             colorscale = colorscale[self.y_axis[str(y_axis_value)]],
            colorscale=self.colorscales[color_column],
            autocolorscale=False,
            reversescale=False,
            marker_line_color='darkgray',
            marker_line_width=0.5,
            colorbar_title=color_column,

            zmax=zmax,
            zmin=zmin
        ))

        fig.update_geos(
            projection_scale=4.5,
            center={'lat': 52, 'lon': 11.977321}
        )

        fig.update_layout(
            title_x=0.5,
            
            width=1200,
            height=500,
            legend={
                "borderwidth": 0
            },
            
            plot_bgcolor = self.colors['secondary'],
            paper_bgcolor = self.colors['secondary'],
            font_color = self.colors['text'],

            title_text= self.y_axis[str(y_axis_value)] + ' in EU countries in ' + str(year) +
            "<br><sup>(Click a country to display a pie chart of it's energy generation distribution)</sup>",
            geo=dict(
                showframe=False,
                showcoastlines=False,
                projection_type='equirectangular'
            )
        )
        return fig

    def update_line_plot(self, y_axis_value, scale):
        value_column = self.y_axis[str(y_axis_value)] if scale == 0 else (self.y_axis[str(y_axis_value)] + ' (log_10)')
        fig = px.line(self.energy_per_area, x="Year", y=value_column, color='Area')
        
        fig.update_layout(
            title_text = "Evolution of '" + value_column.lower() + "' in EU countries" +
            "<br><sup>(Double click on a country to isolate it's data)</sup>",
            title_x=0.5,
            
            width=1200,
            height=300,
            plot_bgcolor = self.colors['secondary'],
            paper_bgcolor = self.colors['secondary'],
            font_color = self.colors['text']
        )
        
        return fig


    def update_sunburst_country(self, clickData, year):
        area = 'France' if clickData == None else clickData['points'][0]['text']
        unwanted_variables = ['Demand', 'Production',
                              'Net imports', 'Fossil', 'Renewables', 'Wind and solar']
        selected_data = self.df[(self.df['Year'] == year)
                                & (self.df['Area'] == area)]
        selected_data = selected_data[~selected_data['Variable'].isin(
            unwanted_variables)]

        fig = px.sunburst(selected_data, path=['Kind of energy', 'Variable'],
                          values='Generation (TWh)',
                          hover_name='Variable',
                          hover_data=['Generation (TWh)', 'Change on last year (%)', 'Change on last year (TWh)'],
                          width=450, height=400,
                          color_discrete_map={'Nuclear':'gold', 'Renewables': '#32CD32'}
                         )
#         fig2 = go.Figure(go.Sunburst(
#             branchvalues = 'total',
#             labels=fig['data'][0]['labels'].tolist(),
#             parents=fig['data'][0]['parents'].tolist(),
#             values=fig['data'][0]['values'].tolist(),
#             ids=fig['data'][0]['ids'].tolist()
#                                     )
#                     )
        fig.update_layout(
            title_text = 'Energy generation in ' + area + ' ' + 'in ' + str(year) +
            "<br><sup>(Click another country on the map to display it's energy generation distribution)</sup>",
            title_x=0.5,
            plot_bgcolor = self.colors['secondary'],
            paper_bgcolor = self.colors['secondary'],
            font_color = self.colors['text']
        )

        return fig

    def update_sunburst_europe(self, year):
        area = 'EU27+1'
        unwanted_variables = ['Demand', 'Production',
                              'Net imports', 'Fossil', 'Renewables', 'Wind and solar']
        selected_data = self.df[(self.df['Year'] == year)
                                & (self.df['Area'] == area)]
        selected_data = selected_data[~selected_data['Variable'].isin(
            unwanted_variables)]
        fig = px.sunburst(selected_data, path=['Kind of energy', 'Variable'],
                          values='Generation (TWh)',
                          hover_name='Variable',
                          hover_data=['Generation (TWh)', 'Change on last year (%)', 'Change on last year (TWh)'],
                          title = 'Energy generation in EU in ' + str(year),
                          width=450, height=400,
                          color_discrete_map={'Fossil':'gold', 'Renewables': '#32CD32'}
                         )
        
        fig.update_layout(
            title_x=0.5,
            plot_bgcolor = self.colors['secondary'],
            paper_bgcolor = self.colors['secondary'],
            font_color = self.colors['text']
        )
        
        return fig

    def run_movie(self, text):
        if text == self.START:    # then it means we are stopped
            return 0
        else:
            return -1

        # start and stop the movie
    def button_on_click(self, n_clicks, text):
        if text == self.START:
            self.is_autoslider_activated = False
            return self.STOP
        else:
            self.is_autoslider_activated = True
            return self.START

    # see if it should move the slider for simulating a movie
    def on_interval(self, n_intervals, year, text):
        if text == self.STOP:  # then we are running
            if year == self.years[-1]:
                return self.years[0]
            else:
                return year + 1
        else:
            return year  # nothing changes

    def update_country_name(self, clickData):
        input_value = 'France' if clickData == None else clickData['points'][0]['text']
        return f"Distribution of Energy Type in {input_value}"

    def run(self, debug=False, port=8050):
        self.app.run_server(host="0.0.0.0", debug=debug, port=port)
        self.update_sunburst_country(None, 2020)
        self.update_sunburst_europe(2020)
        self.update_line_plot(0)
        self.create_map(2000, 4)


if __name__ == '__main__':
    eeg = EuropeEnergyGeneration()
    eeg.run(port=8056, debug=False)


Overwriting /tmp/dash1.py


In [105]:
!python3 /tmp/dash1.py 

Dash is running on http://0.0.0.0:8056/

 * Serving Flask app 'dash1' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses.
 * Running on http://192.168.1.33:8056/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2022 16:45:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2022 16:45:07] "POST /_dash-update-component HTTP/1.1" 200 -
/home/charli/.local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/charli/.local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

127.0.0.1 - - [27/Apr/2022 16:45:08] "POST /_dash-update-component HTTP/1.1" 200 -
/home/charli/.local/lib/python3.8/site-packages/plotly/expres